In [ ]:
import os
import wave
import re

from einops import pack, unpack
import numpy as np
import torch
import torchaudio
from encodec import EncodecModel
from encodec.utils import convert_audio
from transformers import HubertModel
from customtokenizer import CustomTokenizer

from bark_conference import *
from Langchain_conference import *
from matching import *
import custom_voice

In [8]:
# 전역 변수 선언

narrator_path = None
narrator_type = None
custom_path = None
langchain_result = None
voices = None
button_if = False
scenario = None

In [9]:
# npz 생성 관련 model load
def load_model(narr=True):
    device='cuda'

    print('Loading HuBERT...')
    hubert_model = HubertModel.from_pretrained("team-lucid/hubert-base-korean")
    hubert_model.to(device)

    print('Loading Quantizer...')
    if narr: 
        quant_model = CustomTokenizer.load_from_checkpoint("../KOR-HuBERT-Quantizer/Literature/new_model_epoch_37.pth", device)
    else:
        quant_model = CustomTokenizer.load_from_checkpoint("../KOR-HuBERT-Quantizer/Literature/new_model_epoch_8.pth", device)
        

    print('Loading Encodec...')
    encodec_model = EncodecModel.encodec_model_24khz()
    encodec_model.set_target_bandwidth(6.0)
    encodec_model.to(device)

    print('Downloaded and loaded models!')
    return hubert_model, quant_model, encodec_model

In [ ]:
# LangChain 실행
def process_text(novel_text):
    global langchain_result
    langchain_result = langchain(novel_text)
    # 결과를 문자열로 포맷팅
    characters_str = ", ".join([f"{name}:{attributes}" for name, attributes in langchain_result['Characters'].items()])
    scenario_str = "\n".join([f"{item[0]}: '{item[1]}', {item[2]}" if len(item) == 3 else f"{item[0]}: '{item[1]}'" for item in langchain_result['Scenario']])

    formatted_result = f"Characters:\n{characters_str}\n\nScenario:\n{scenario_str}"
    return formatted_result

novel_text = "여기에 소설 텍스트를 입력합니다."

# Langchain 결과 처리 및 출력
langchain_result = process_text(novel_text)
print(langchain_result)


In [ ]:
# 파일 업로드 후 처리를 위한 함수 정의
def process_uploaded_file(change):
    upload = change.owner
    if upload.data:
        uploaded_filename = next(iter(upload.value))  # 업로드된 파일 이름 가져오기
        content = upload.data[0]  # 업로드된 파일의 내용

        # 저장할 경로 설정
        save_directory = '../demo/voice/custom/'
        save_path = os.path.join(save_directory, 'uploaded_voice.wav')

        # 파일 저장
        with open(save_path, 'wb') as f:
            f.write(content)

        # 모델 로딩 및 음성 처리 (가정)
        hubert_model, quant_model, encodec_model = load_model()
        global custom_path
        custom_path = create_voice(save_path, hubert_model, quant_model, encodec_model)
        
        # 처리 완료 메시지 표시
        clear_output()
        print(f'파일이 처리되었습니다: {custom_path}')
    else:
        clear_output()
        print("파일이 업로드되지 않았습니다.")

upload = FileUpload(accept='.wav', multiple=False)
upload.observe(process_uploaded_file, names='data')

# 위젯 표시
display(upload)

In [ ]:
# 내레이터 선택 및 로드 처리 함수
def process_narrative_selection(gender, age):
    global narrator_type
    
    filename = f"{gender}_{age}.npz" 
    file_path = f"../demo/voice/narrator/{filename}"
    
    try:
        # npz 파일 로드
        data = np.load(file_path)
        # 로드된 데이터를 전역 변수에 저장
        narrator_type = data
        print(f"'{filename}' 파일이 성공적으로 로드되었습니다.")
        
    except FileNotFoundError:
        print(f"'{file_path}' 파일을 찾을 수 없습니다.")

# 둘 중에 하나를 골라서 사용하세요
process_narrative_selection('남', '성인')  # '여', '성인'

In [ ]:
def matching_algorithm():
    global narrator_type
    global narrator_path
    global custom_path
    global langchain_result
    global scenario
    global voices

    # 나레이터 경로 설정
    if custom_path:
        narrator_path = custom_path
    else:
        narrator_path = matching.preset_narr(narrator_type[0], narrator_type[1], False)

    # langchain 결과에서 캐릭터 정보와 시나리오 추출
    characters = langchain_result['Characters']
    scenario = langchain_result['Scenario']

    # 캐릭터별 목소리 매칭
    voices = matching.match_character(characters)
    voices['Narrator'] = narrator_path

    # 매칭 결과 출력
    print("매칭된 목소리 파일 경로:")
    for character, voice_path in voices.items():
        print(f"{character}: {voice_path}")

# matching_algorithm 함수 실행
matching_algorithm()

In [ ]:
# 오디오 생성 및 결합
def full_bark_wrapper():
    matching_algorithm()

    if scenario is None or voices is None:
        print("scenario 또는 voices가 설정되지 않았습니다.")
        return {"audios": []}

    full_bark(scenario, voices)
    audio_book = combine('../demo/voice/audio_book/result')

full_bark_wrapper()